In [ ]:
# Lucas Calzada del Pozo
# Victoria Martin Rojo
import numpy as np   
import matplotlib.pyplot as plt
import sounddevice as sd
from consts import *         
import tkinter as tk 
from slider import *         
from adsr import *           
from synthFM import *         
import mido

# ------------------------------------------------------------
# Global: Diccionario que contendrá todos los instrumentos
global_instruments = {}

# Callback del stream de audio: mezcla la señal de todos los instrumentos.
def callback(outdata, frames, time, status):
    if status:
        print(status)
    out = np.zeros(CHUNK)
    for inst in global_instruments.values():
        out += inst.next()
    s = np.float32(out)
    outdata[:] = s.reshape(-1, 1)

stream = sd.OutputStream(samplerate=SRATE, callback=callback, blocksize=CHUNK)
stream.start()

# ------------------------------------------------------------
class Instrument:
    def __init__(self, master, name="FM synthesizer", amp=0.2, ratio=3, beta=0.6):    
        # Panel del sintetizador usando 'master'
        frame = tk.LabelFrame(master, text=name, bg="#808090")
        frame.pack(side=tk.LEFT)
        # Controles del oscilador FM
        frameOsc = tk.LabelFrame(frame, text="FM oscillator", bg="#808090")
        frameOsc.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        
        self.ampS = Slider(frameOsc, 'amp', packSide=tk.TOP,
                           ini=amp, from_=0.0, to=1.0, step=0.05) 
        self.ratioS = Slider(frameOsc, 'ratio', packSide=tk.TOP,
                           ini=ratio, from_=0.0, to=20.0, step=0.5)
        self.betaS = Slider(frameOsc, 'beta', packSide=tk.TOP,
                            ini=beta, from_=0.0, to=10.0, step=0.05) 
        
        # Ventana de texto para lanzar notas con el teclado
        text = tk.Text(frameOsc, height=4, width=40)
        text.pack(side=tk.BOTTOM)
        text.bind('<KeyPress>', self.down)
        text.bind('<KeyRelease>', self.up)

        # Controles ADSR
        frameADSR = tk.LabelFrame(frame, text="ADSR", bg="#808090")
        frameADSR.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        self.attackS = Slider(frameADSR, 'attack', ini=0.01, from_=0.0, to=0.5,
                              step=0.005, orient=tk.HORIZONTAL, packSide=tk.TOP) 
        self.decayS = Slider(frameADSR, 'decay', ini=0.01, from_=0.0, to=0.5,
                             step=0.005, orient=tk.HORIZONTAL, packSide=tk.TOP)
        self.sustainS = Slider(frameADSR, 'sustain', ini=0.4, from_=0.0, to=1.0,
                               step=0.01, orient=tk.HORIZONTAL, packSide=tk.TOP) 
        self.releaseS = Slider(frameADSR, 'release', ini=0.5, from_=0.0, to=4.0,
                               step=0.05, orient=tk.HORIZONTAL, packSide=tk.TOP) 
        
        # Diccionarios para las notas activas (canales) y para los "tails" de fade-out.
        self.channels = dict()        
        self.tails = dict()
    
    def getConfig(self):
        return (self.ampS.get(), self.ratioS.get(), self.betaS.get(),
                self.attackS.get(), self.decayS.get(), self.sustainS.get(),
                self.releaseS.get())

    def noteOn(self, midiNote):
        # Si la nota ya está activa, aplicamos un fade-out y la movemos a tails.
        if midiNote in self.channels:                   
            lastAmp = self.channels[midiNote].adsr.last
            env = Env([(0, lastAmp), (CHUNK, 0)]).next()
            signal = self.channels[midiNote].next()
            self.tails[midiNote] = env * signal
        # Obtenemos la frecuencia a partir del número MIDI y creamos un nuevo synth.
        freq = freqsMidi[midiNote]
        self.channels[midiNote] = SynthFM(fc=freq, amp=self.ampS.get(), ratio=self.ratioS.get(),
                                          beta=self.betaS.get(), attack=self.attackS.get(),
                                          decay=self.decayS.get(), sustain=self.sustainS.get(),
                                          release=self.releaseS.get())

    def noteOff(self, midiNote):
        if midiNote in self.channels:
            self.channels[midiNote].noteOff()

    def down(self, event):
        c = event.keysym
        if c == '0': 
            self.stop()            
        elif c in teclas:
            midiNote = 48 + teclas.index(c)
            print(f'noteOn {midiNote}')
            self.noteOn(midiNote)
            
    def up(self, event):
        c = event.keysym
        if c in teclas:
            midiNote = 48 + teclas.index(c)
            print(f'noteOff {midiNote}')
            self.noteOff(midiNote)

    def next(self):
        out = np.zeros(CHUNK)
        for c in list(self.channels):
            if self.channels[c].state == 'off':
                del self.channels[c]
            else:
                if c in self.tails:
                    out += self.tails[c]
                    del self.tails[c]
                else:
                    out += self.channels[c].next()
        return out

    def stop(self):
        self.channels = dict()

# ------------------------------------------------------------
# Secuenciador MIDI que utiliza varios instrumentos (uno por canal)
class MidiSequencerTk:
    def __init__(self, master):
        self.tk = master
        # Creamos un instrumento para cada canal MIDI 
        self.instruments = {}
        for ch in range(3):
            beta = 0.6 + 0.1 * ch
            self.instruments[ch] = Instrument(master, name=f"FM Synth Ch {ch}", amp=0.2, ratio=3, beta=beta)
        # Actualizamos la variable global que utiliza el callback de audio para mezclar.
        global global_instruments
        global_instruments = self.instruments

        # Interfaz del secuenciador MIDI
        frame = tk.LabelFrame(master, text="Secuenciador MIDI", bg="#908060")
        frame.pack(side=tk.TOP)
        frameFile = tk.Frame(frame, highlightbackground="blue", highlightthickness=6)
        frameFile.pack(side=tk.TOP)
        tk.Label(frameFile, text='Archivo MIDI: ').pack(side=tk.LEFT)
        self.file = tk.Entry(frameFile)
        self.file.insert(14, "pirates.mid")
        self.file.pack(side=tk.LEFT)
        self.transport = 0
        
        self.text = tk.Text(frame, height=6, width=23)
        self.text.pack(side=tk.RIGHT)
        playBut = tk.Button(frame, text="Play", command=self.play)
        playBut.pack(side=tk.TOP)
        stopBut = tk.Button(frame, text="Stop", command=self.stop)
        stopBut.pack(side=tk.BOTTOM)

        self.tick = 1  # intervalo en milisegundos
        self.state = 'off'

    def getSeq(self, midiEvents):
        seq = []
        accTime = 0
        for m in midiEvents:
            accTime += m.time
            if m.type == 'note_on':
                if m.velocity == 0:
                    seq.append((accTime, 'noteOff', m.note + self.transport, m.channel))
                else:
                    seq.append((accTime, 'noteOn', m.note + self.transport, m.channel))
            elif m.type == 'note_off':
                seq.append((accTime, 'noteOff', m.note + self.transport, m.channel))
        return seq

    def play(self):
        events = mido.MidiFile(self.file.get())
        seq = self.getSeq(events)
        print(seq)
        self.state = 'on'
        self.playLoop(seq)

    def playLoop(self, seq, item=0, accTime=0):
        if item >= len(seq) or self.state == 'off':
            return
        # Procesa todos los eventos cuyo tiempo sea menor o igual al tiempo acumulado.
        while item < len(seq) and accTime >= seq[item][0]:
            (_, msg, midiNote, ch) = seq[item]
            self.text.insert('6.0', f'ch {ch}: {msg} {midiNote}\n')
            if msg == 'noteOn':
                if ch in self.instruments:
                    self.instruments[ch].noteOn(midiNote)
                else:
                    self.instruments[ch] = Instrument(self.tk, name=f"FM Synth Ch {ch}")
                    self.instruments[ch].noteOn(midiNote)
            else:  # noteOff
                if ch in self.instruments:
                    self.instruments[ch].noteOff(midiNote)
            item += 1

        accTime += self.tick / 1000.0
        self.text.after(self.tick, lambda: self.playLoop(seq, item, accTime))

    def stop(self):
        for inst in self.instruments.values():
            inst.stop()
        self.state = 'off'

# ------------------------------------------------------------
# Programa principal
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Multi-Instrument MIDI Sequencer")
    sequencer = MidiSequencerTk(root)
    root.mainloop()


[(0, 'noteOn', 62, 0), (0.24999999999999997, 'noteOff', 62, 0), (0.3, 'noteOn', 62, 0), (0.44687499999999997, 'noteOff', 62, 0), (0.44999999999999996, 'noteOn', 62, 0), (0.7, 'noteOff', 62, 0), (0.75, 'noteOn', 62, 0), (0.896875, 'noteOff', 62, 0), (0.9, 'noteOn', 62, 0), (1.15, 'noteOff', 62, 0), (1.2, 'noteOn', 62, 0), (1.3468749999999998, 'noteOff', 62, 0), (1.3499999999999999, 'noteOn', 62, 0), (1.4968749999999997, 'noteOff', 62, 0), (1.4999999999999998, 'noteOn', 62, 0), (1.6468749999999996, 'noteOff', 62, 0), (1.6499999999999997, 'noteOn', 62, 0), (1.7968749999999996, 'noteOff', 62, 0), (1.7999999999999996, 'noteOn', 62, 0), (2.0499999999999994, 'noteOff', 62, 0), (2.099999999999999, 'noteOn', 62, 0), (2.2468749999999993, 'noteOff', 62, 0), (2.249999999999999, 'noteOn', 62, 0), (2.499999999999999, 'noteOff', 62, 0), (2.549999999999999, 'noteOn', 62, 0), (2.696874999999999, 'noteOff', 62, 0), (2.699999999999999, 'noteOn', 62, 0), (2.949999999999999, 'noteOff', 62, 0), (2.999999999